In [1]:
# Program symuluje "na żywo" zachowanie gazu oddziałującym z potencjałem Leonarda-Jonesa w róznych temperaturach. 

# Można zaobserwować że dla niskich temperatur cząsteczki gazu zbierają się w grupę i nie wykonują zbyt znaczących 
# ruchów. Odpowiada to przejściu symulowanej sybstancji do stałego stanu skupienia. 

# Dla nieco wyższych temperatur cząsteczki poruszają się o wiele swobodniej, jednak oddziaływania pomiędzy nimi mają 
# ciągle znaczący wpływ na ich ruch. Odpowiada to przejściu symulowanej sybstancji do ciekłego  stanu skupienia. 

# Dla wysokich temperatur oddziaływania pomiędzy cząsteczkami przestają być znaczące, a cząsteczki zaczynają przekazywac 
# sobie energię głównie poprzez zderzenia. Wtedy symulowana substancja zaochowuje się jak gaz.

# Po odpaleniu wszystkich komórekw pliku powinno wyświetlić się okno animacji. Po jej odpaleniu suwakiem znajdującym 
# się po prawej stronie okna moża regulować temperaturę gazu (W tym przypadku nie jest ona opisana w jednostkach SI). 

# Przejście do zbyt wysokich temperatur (ustawienie suwaka na maksa do góry) może okazjonalnie prowadzić do zepsucia animacji.
# Jest to spowodowane naturą samej wykorzystanej metody symulacji oraz natury użytego potencjału - jeśli cząsteczki znajdą się
# za blisko siebe to zadziała na nie w praktyce nieskończona siła, co z kolei prowadzi to powstania w układzi zbyt dużych prękości

# Obliczenie wykonywane są dla 16 cząsteczek. Równania ruchu są rozwiązywane przy użuciu metody eulera, a w samym układzie 
# zostały zaimplementowane warunki brzegowe pozwalające skuteczne symulowanie zachowania substancji

In [2]:
# Biblioteka GLMakie potrafi być problematyczna w instalacji. Jeżeli doszłoby do sytuacji w której niebyłby Pan w stanie jej zainstalować
# to jestem w stanie nagrać film prezentujący działanie progamu

using GLMakie
using Random
using Observables

In [3]:
# Poniższe struktury danych okazały się nie być konieczne do zaimplementowania, ale zaimplementowałem w ich oparciu na tyle dużą część
# kodu że postanowiłem ich nie usuwać

In [4]:
mutable struct vecVal
	x::Float64
	y::Float64
end

mutable struct Particle
	pos::vecVal
	speed::vecVal
end

In [5]:
Base.broadcastable(f::vecVal) = Ref(f)

Base.broadcastable(f::Particle) = Ref(f)

function Base.:+(a::vecVal, b::Vector{Float64})
	return vecVal(a.x + b[0], a.y + b[1])
end 

function Base.:+(a::vecVal, b::vecVal)
	return vecVal(a.x + b.x, a.y + b.y)
end

function Base.:-(a::vecVal, b::vecVal)
	return vecVal(a.x - b.x, a.y - b.y)
end

function Base.:*(a::vecVal, b::Number)
	return vecVal(a.x * b, a.y * b)
end

function Base.:*(b::Number, a::vecVal)
	return vecVal(a.x * b, a.y * b)
end

function Base.:/(a::vecVal, b::Number)
	return vecVal(a.x / b, a.y / b)
end

function Base.:+(a::Particle, b::Particle)
	return vecVal(a.pos + b.pos, a.speed + b.speed)
end

function Base.:-(a::Particle, b::Particle)
	return vecVal(a.pos - b.pos, a.speed - b.speed)
end

function Base.:*(a::Particle, b::Number)
	return vecVal(a.pos, a.speed * b)
end

function Base.:*(b::Number, a::Particle)
	return vecVal(a.pos, a.speed * b)
end

function Base.:/(a::Particle, b::Number)
	return vecVal(a.pos, a.speed / b)
end

In [6]:
function get_average_speed(particles::Vector{Particle})
    sum = 0.0
    elems = 0
    for p in particles
        sum += sqrt(p.speed.x^2 + p.speed.y^2)
        elems +=1
    end
    return sum/elems
end

get_average_speed (generic function with 1 method)

In [7]:
function closest_image(p1::vecVal, p2::vecVal, boundary::vecVal)
	dx = p2.x - p1.x
	dy = p2.y - p1.y
	dx = (abs(dx) > boundary.x - abs(dx) ? -sign(dx) * (boundary.x - abs(dx)) : dx)
	dy = (abs(dy) > boundary.y - abs(dy) ? -sign(dy) * (boundary.y - abs(dy)) : dy)

	return [dx, dy]
end

closest_image (generic function with 1 method)

In [8]:
function get_force(p1::Particle, p2::Particle, boundary::vecVal, sigma::Float64, epsilon::Float64)
	if p1.pos == p2.pos
		return vecVal(0.0, 0.0)
	end

	r_vec = closest_image(p1.pos, p2.pos, boundary)
	r = Base.Math._hypot(r_vec...)
	r_vec = r_vec ./ r

	force = (24 * epsilon * sigma^6 * (r^6 - 2 * sigma^6)) / r^13

	return vecVal((r_vec * force)...)
end


get_force (generic function with 1 method)

In [9]:
function get_forces_vector(particles::Vector{Particle},  boundary::vecVal, sigma::Float64, epsilon::Float64)
	function temp(particle)
    	return sum(get_force.(particle, particles,boundary, sigma, epsilon))
	end
	return temp.(particles)
end

get_forces_vector (generic function with 1 method)

In [10]:
function bound(position::vecVal, boundary::vecVal)
	return vecVal(
		((position.x % boundary.x) < 0 ? boundary.x + (position.x % boundary.x) : (position.x % boundary.x)),
		((position.y % boundary.y) < 0 ? boundary.y + (position.y % boundary.y) : (position.y % boundary.y)),
	)
end

bound (generic function with 1 method)

In [11]:
function move_particle(particle::Particle, boundary::vecVal, force::vecVal, sigma::Float64, epsilon::Float64, m::Float64, delta::Float64)
	acc = force ./ m
	particle.speed = particle.speed + delta .* acc
	particle.pos = bound(particle.pos + particle.speed .* delta, boundary)
	return particle
end

move_particle (generic function with 1 method)

In [12]:
function move(particles::Vector{Particle}, boundary::vecVal, sigma::Float64, epsilon::Float64, m::Float64, delta::Float64)
	forces = get_forces_vector(particles, boundary, sigma, epsilon)
	return [move_particle(particle, boundary,force, sigma, epsilon, m, delta) for (particle, force) in zip(particles, forces)]
end


move (generic function with 1 method)

In [13]:
function get_x_and_y(particles)
    return [Point2f(p.pos.x , p.pos.y ) for p in particles]
end

get_x_and_y (generic function with 1 method)

In [14]:
function modify_velocity(particles::Vector{Particle}, e::Float64)
    return [Particle(a.pos, a.speed*e) for a in particles]
end

modify_velocity (generic function with 1 method)

In [15]:
function modify_temperature(particles, temp::Float64)
    particles[] = modify_velocity(particles[], sqrt(temp/get_average_speed(particles[])))
end

modify_temperature (generic function with 1 method)

In [16]:
function iterate_animation(particles, a, boundary ,sigma, epsilon, m, delta)
    particles[] = move(particles[], boundary ,sigma, epsilon, m, delta)
    a[] = get_x_and_y(particles[])
end

iterate_animation (generic function with 1 method)

In [17]:
sigma = 0.6
epsilon = 0.7
m=1.
delta = 0.001
temp = 2.5
boundary = vecVal(5,5)

vecVal(5.0, 5.0)

In [18]:
rng = MersenneTwister(1234);

particles = Observable(vec(reshape([Particle(vecVal(pos...), vecVal((randn!(rng, zeros(2)))...)) for pos in Iterators.product(1:4, 1:4) |> collect],(1,16))))

a = Observable(get_x_and_y(particles[]))


modify_temperature(particles,temp)


fig = Figure(resolution = (850, 800))

ax = Axis(fig[1, 1], aspect = 1)

sl_y = Slider(fig[1, 2], range = 0:0.01:25, horizontal = false, startvalue = temp)

scatter!(ax,a, marker = :circle,
color = :purple, markersize = 50*sqrt(2)*sigma/2^6, markerspace = :data)

limits!(ax, 0, boundary.x, 0, boundary.y)
fig

In [19]:
l1 = lift(sl_y.value) do val
    modify_temperature(particles,val)
end

text = Observable("start")

run = Button(fig[2,1]; label = text, tellwidth = false)
isrunning = Observable(false)
on(run.clicks) do clicks 
    isrunning[] = !isrunning[]; 
    if !isrunning[]
        text[] = "start"
    else
        text[] = "pause"
    end
end
on(run.clicks) do clicks

    @async while isrunning[]
        isopen(fig.scene) || break 
        iterate_animation(particles, a, boundary ,sigma, epsilon, m, delta)
        sleep(0.0001)
    end
end

ObserverFunction defined at c:\Users\wojte\OneDrive\Pulpit\projekt.ipynb:19 operating on Observable{Any}(0)